In [ ]:
# Colab: Gerar planilha com links de itens (arquivos e pastas) + QR-Codes

# 0) Instalação das dependências (rodar só na 1ª vez)
# !pip install -q qrcode[pil] pandas openpyxl XlsxWriter google-api-python-client

# 1) Monte seu Drive e autentique
from google.colab import drive, auth, files as gfiles
import os, re, shutil
import qrcode
import pandas as pd
from PIL import Image, ImageDraw, ImageFont
from googleapiclient.discovery import build

drive.mount('/content/drive', force_remount=True)
auth.authenticate_user()

# 2) Construa o serviço da Drive API
service = build('drive', 'v3')

# 3) Defina o ID da pasta-mãe (substitua pelo ID correto)
parent_id = 'ID_AQUI'  # ajuste conforme necessário

# 4) Liste todos os itens dentro da pasta-mãe (arquivos e pastas)
resp = service.files().list(
    q=f"'{parent_id}' in parents and trashed=false",
    fields='files(id,name,mimeType)',
    includeItemsFromAllDrives=True,
    supportsAllDrives=True
).execute()

# Debug: confira o retorno da API
items = resp.get('files', [])
print(f'>>> itens encontrados na pasta-mãe: {len(items)}')
for f in items:
    print(f" - {f['name']} [{f['mimeType']}] ({f['id']})")

# 5) Abortar se não houver itens
if not items:
    print('🚨 Nenhum item encontrado. Verifique o parent_id ou permissões.')
    raise SystemExit

# 6) Crie registros com link público para cada item
registros = []
for f in items:
    fid, name, mime = f['id'], f['name'], f['mimeType']
    # ajuste da URL conforme tipo (pasta vs. arquivo)
    if mime == 'application/vnd.google-apps.folder':
        link = f'https://drive.google.com/drive/folders/{fid}?usp=sharing'
    else:
        link = f'https://drive.google.com/file/d/{fid}/view?usp=sharing'
    # garante permissão de leitura para anyone
    perms = service.permissions().list(
        fileId=fid,
        fields='permissions(type,role)'
    ).execute().get('permissions', [])
    if not any(p['type']=='anyone' for p in perms):
        service.permissions().create(
            fileId=fid,
            body={'type':'anyone','role':'reader'}
        ).execute()
    registros.append({'Nome': name, 'Link': link})

# 7) Converta em DataFrame e confira
df = pd.DataFrame(registros)
print(f'>>> registros no DataFrame: {len(df)}')

# 8) Geração de QR-Codes
DIR_OUT = '/content/qrcodes'
os.makedirs(DIR_OUT, exist_ok=True)
font = ImageFont.load_default()
df['QR_Path'] = ''
for idx, row in df.iterrows():
    texto, nome = row['Link'], row['Nome']
    qr = qrcode.make(texto).convert('RGB')
    w, h = qr.size
    padding = 20
    canvas = Image.new('RGB', (w, h + padding), 'white')
    canvas.paste(qr, (0, 0))
    draw = ImageDraw.Draw(canvas)
    bbox = draw.textbbox((0, 0), nome, font=font)
    tw, th = bbox[2]-bbox[0], bbox[3]-bbox[1]
    x = (w - tw)//2
    y = h + (padding - th)//2
    draw.text((x, y), nome, fill='black', font=font)
    safe = re.sub(r'[\\/*?:"<>|]', '', nome).replace(' ', '_')
    img_file = os.path.join(DIR_OUT, f"{safe}.png")
    canvas.save(img_file)
    df.at[idx, 'QR_Path'] = img_file
    print(f'✔️  {nome}')

# Verifique conteúdo do diretório de saída
files_qr = os.listdir(DIR_OUT)
print('Arquivos em', DIR_OUT, ':', files_qr)
if not files_qr:
    print('🚨 Nenhum QR-Code foi gerado. Abortando.')
    raise SystemExit

# 9) Crie a planilha Excel com imagens embutidas
OUT_XLSX = 'planilha_qr_codes.xlsx'
with pd.ExcelWriter(OUT_XLSX, engine='xlsxwriter') as writer:
    df.to_excel(writer, sheet_name='QR_Codes', index=False, startrow=1)
    ws = writer.sheets['QR_Codes']
    col_qr = df.columns.get_loc('QR_Path')
    for row_num, img in enumerate(df['QR_Path'], start=2):
        ws.insert_image(row_num-1, col_qr, img, {'x_scale':0.7,'y_scale':0.7,'object_position':2})
print(f'\n✅ Planilha "{OUT_XLSX}" criada em /content')

# 10) Compacte todos os QR-PNGs em um ZIP
zip_name = 'qr_codes'
zip_path = shutil.make_archive(zip_name, 'zip', DIR_OUT)
print(f'✅ Arquivo {zip_path} gerado')

# 11) Faça o download dos arquivos
gfiles.download(OUT_XLSX)
gfiles.download(zip_path)
